In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import urllib3

In [2]:
# SSL 경고 비활성화
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

In [3]:
# 1. 웹페이지 가져오기
url = "https://news.seoul.go.kr/traffic/archives/1551"
response = requests.get(url, verify=False)  # get 메소드 사용 및 SSL 인증서 검증 비활성화
print("문제 1. 웹페이지 가져오기")
print("답: 상태 코드:", response.status_code)  # status_code 속성을 사용하여 상태 코드를 출력합니다.

문제 1. 웹페이지 가져오기
답: 상태 코드: 200


In [5]:
# 2. 모든 테이블 찾기
soup = BeautifulSoup(response.text, 'html.parser')  # HTML 파서를 지정합니다.
tables = soup.select('table')  # 모든 <table> 태그를 찾습니다.
print("\n문제 2. 모든 테이블 찾기")
print(f"답: 찾은 테이블 수: {len(tables)}")  # tables의 길이를 출력합니다.


문제 2. 모든 테이블 찾기
답: 찾은 테이블 수: 3


In [9]:
tables

[<table style="height: 882px;" width="1408">
 <tbody>
 <tr>
 <td rowspan="2">
 <p>구 분</p>
 </td>
 <td rowspan="2">
 <p>계</p>
 </td>
 <td rowspan="2">
 <p>1호선</p>
 </td>
 <td colspan="3">
 <p>2 호 선</p>
 </td>
 <td rowspan="2">
 <p>3호선</p>
 </td>
 <td rowspan="2">
 <p>4호선</p>
 </td>
 <td rowspan="2">
 <p>5호선</p>
 </td>
 <td rowspan="2">
 <p>6호선</p>
 </td>
 <td rowspan="2">
 <p>7호선</p>
 </td>
 <td rowspan="2">
 <p>8호선</p>
 </td>
 <td rowspan="2">
 <p>9호선</p>
 </td>
 <td rowspan="2">
 <p>우이</p>
 <p>신설선</p>
 </td>
 <td rowspan="2">
 <p>신림선</p>
 </td>
 </tr>
 <tr>
 <td>
 <p>순 환</p>
 </td>
 <td colspan="2">
 <p>지 선</p>
 </td>
 </tr>
 <tr>
 <td>
 <p>구 간</p>
 </td>
 <td>
 <p>11개</p>
 </td>
 <td>
 <p>서울역</p>
 <p>~</p>
 <p>청량리</p>
 </td>
 <td>
 <p>성수</p>
 <p>~</p>
 <p>성수</p>
 </td>
 <td>
 <p>신설동</p>
 <p>~</p>
 <p>성수</p>
 </td>
 <td>
 <p>신도림</p>
 <p>~</p>
 <p>까치산</p>
 </td>
 <td>
 <p>지축</p>
 <p>~</p>
 <p>오금</p>
 </td>
 <td>
 <p>당고개</p>
 <p>~</p>
 <p>남태령</p>
 </td>
 <td>
 <p>방화</p>
 <p>~</p>
 <p>하남

In [ ]:
# 3. 각 테이블 처리
subway_data = {}
for i, table in enumerate(tables):
    title = table.find_previous_sibling('h5')  # 테이블 앞의 <h5> 태그를 찾습니다.
    table_name = title.get_text().strip() if title else f"Table {i + 1}"  # 테이블 이름을 설정합니다.

    data = [[td.get_text().strip() for td in tr.select('th, td')] for tr in table.select('tr')]  # 테이블의 모든 셀 내용을 추출합니다.
    max_cols = max(len(row) for row in data)  # 가장 긴 행의 길이를 찾습니다.
    data = [row + [''] * (max_cols - len(row)) for row in data]  # 모든 행의 길이를 동일하게 맞춥니다.

    df = pd.DataFrame(data[1:], columns=data[0])  # DataFrame을 생성합니다.
    subway_data[table_name] = df

print("\n문제 3. 테이블 데이터 추출 및 DataFrame 생성")
print(f"답: 처리된 테이블 수: {len(subway_data)}")  # 처리된 테이블의 수를 출력합니다.


문제 3. 테이블 데이터 추출 및 DataFrame 생성
답: 처리된 테이블 수: 3


In [17]:
# 4. 결과 출력
print("\n문제 4. 각 테이블의 요약 정보 출력")
for name, df in subway_data.items():  # 각 테이블에 대해 반복합니다.
    print(f"\n테이블 이름: {name}")
    print("답: 처음 5개 행")
    print(df.head())  # 각 DataFrame의 처음 5개 행을 출력합니다.
    print("\n" + "=" * 50)



문제 4. 각 테이블의 요약 정보 출력

테이블 이름: 
답: 처음 5개 행
          구 분            계          1호선      2 호 선         3호선          4호선  \
0         순 환          지 선                                                    
1         구 간          11개  서울역\n~\n청량리  성수\n~\n성수  신설동\n~\n성수  신도림\n~\n까치산   
2  영업거리\n(km)     359.\n86          7.8       48.8         5.4            6   
3         역 수          338           10         43           4            3   
4   소요시간\n(분)  9\n~\n109.5           18         90           9           11   

         5호선          6호선                  7호선        8호선        9호선  \
0                                                                      
1  지축\n~\n오금  당고개\n~\n남태령  방화\n~\n하남\n검단산\n/마천  응암\n~\n신내  장암\n~\n온수   
2       38.2         31.7                 59.8       36.3       46.9   
3         34           26                   56         39         42   
4       67.5           53                109.5       73.8         87   

             우이\n신설선                     신림선    

In [18]:
# 5. 첫 번째 테이블에서 노선별 정보 추출
print("\n문제 5. 첫 번째 테이블에서 노선별 정보 추출")
if subway_data:
    first_table_name = list(subway_data.keys())[0]  # 첫 번째 테이블의 이름을 가져옵니다.
    operation_data = subway_data[first_table_name]  # 첫 번째 테이블의 데이터를 가져옵니다.

    print("답: 노선별 정보")
    for _, row in operation_data.iterrows():  # 각 행에 대해 반복합니다.
        if row.iloc[0] and row.iloc[0] != '구 분':  # 첫 번째 열이 비어있지 않고 '구 분'이 아닌 경우
            print(f"\n{row.iloc[0]}:")  # 노선 이름을 출력합니다.
            for col, val in row.items():  # 각 열에 대해 반복합니다.
                if col != row.iloc[0] and val:  # 첫 번째 열이 아니고 값이 있는 경우
                    print(f"  {col}: {val}")  # 열 이름과 값을 출력합니다.



문제 5. 첫 번째 테이블에서 노선별 정보 추출
답: 노선별 정보

순 환:
  구 분: 순 환
  계: 지 선

구 간:
  구 분: 구 간
  계: 11개
  1호선: 서울역
~
청량리
  2 호 선: 성수
~
성수
  3호선: 신설동
~
성수
  4호선: 신도림
~
까치산
  5호선: 지축
~
오금
  6호선: 당고개
~
남태령
  7호선: 방화
~
하남
검단산
/마천
  8호선: 응암
~
신내
  9호선: 장암
~
온수
  우이
신설선: 암사
역사
공원
~
모란
  신림선: 개화
~
중앙
보훈
병원
  : 북한산
우이
~
신설동
  : 샛강
~
관악산

영업거리
(km):
  구 분: 영업거리
(km)
  계: 359.
86
  1호선: 7.8
  2 호 선: 48.8
  3호선: 5.4
  4호선: 6
  5호선: 38.2
  6호선: 31.7
  7호선: 59.8
  8호선: 36.3
  9호선: 46.9
  우이
신설선: 19.2
  신림선: 40.6
  : 11.4
  : 7.76

역 수:
  구 분: 역 수
  계: 338
  1호선: 10
  2 호 선: 43
  3호선: 4
  4호선: 3
  5호선: 34
  6호선: 26
  7호선: 56
  8호선: 39
  9호선: 42
  우이
신설선: 19
  신림선: 38
  : 13
  : 11

소요시간
(분):
  구 분: 소요시간
(분)
  계: 9
~
109.5
  1호선: 18
  2 호 선: 90
  3호선: 9
  4호선: 11
  5호선: 67.5
  6호선: 53
  7호선: 109.5
  8호선: 73.8
  9호선: 87
  우이
신설선: 51.5
  신림선: 79.8
(일반)
52.5
(급행)
  : 23
  : 18.1

운행시격
(분):
  구 분: 운행시격
(분)
  계: RH
  1호선: 3.0
  2 호 선: 2.5
  3호선: 7.0
  4호선: 10.0
  5호선: 3.0
  6호선: 2.5
  7호선: 2.5
  8호선: 4.0
  9호선: 2.5
  우